# Library

In [133]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import re
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

## chrome driver 설치

In [134]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [135]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [136]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [207]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

## 페이지 접속

In [200]:
# 페이지 주소
url = 'https://shopping.naver.com/ns/home'

### requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [201]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

### selenium 작동

In [208]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

### 검색 창에 검색어 입력

In [209]:
search_keyword = '아이라인'
# search_window_css = 'div._gnbSearch_gnb_search_ULxKx input' # 검색 입력창
search_window_xpath = '//*[@id="gnb-gnb"]/div[2]/div/div[2]/div[1]/form/div/div/div/div/input' # 검색 입력창
# search_window_xpath2= '//*[@id="gnb-gnb"]/div[2]/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input' # 키워드 입력 후 검색 입력창
# search_window = driver.find_element(By.CSS_SELECTOR, search_window_css)
search_window = driver.find_element(By.XPATH, search_window_xpath)
search_window.send_keys(search_keyword)
time.sleep(random.randrange(1, 3))

In [210]:
# search_window2 = driver.find_element(By.XPATH, search_window_xpath2)
search_window.send_keys(Keys.ENTER)
driver.implicitly_wait(10)

In [211]:
# 스크롤해야 정보가 로드되는 페이지
# 페이지 스크롤 다운
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 문서 높이 만큼 스크롤

# 무한 페이지 스크롤다운
# last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

# while True:
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 문서 높이 만큼 스크롤
#     # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") # # 문서 높이 만큼 스크롤(위 코드와 동일한 기능)
#     time.sleep(random.uniform(1, 3))
#     driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료

#     new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
#     if new_page_height == last_page_height:
#         break
#     else:
#         last_page_height = new_page_height

## 🔧 입력 키워드 정보 수집
- 페이지 로드되어도 보고있는 화면 근처 정보만 수집되는걸로 보임
    1. 페이지 스크롤 -> 수집 -> 페이지 스크롤 반복
    2. 수집 후 중복 제거

In [212]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'아이라인 : 네이버+ 스토어 검색'

In [213]:
# 상품 섹션
products_section = soup.select_one('div#composite-card-list')
len(products_section)

2

In [214]:
# 상품 리스트
product_list = products_section.select('li.compositeCardContainer_composite_card_container__jr8cb.composite_card_container div.basicProductCard_basic_product_card__TdrHT.basicProductCard_view_type_grid2__vKr1n')
len(product_list)

99

### 상세 페이지 링크

In [215]:
# 상품 상세 페이지
order_number = 0
product_list[order_number].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href']

'https://adcr.naver.com/adcr?x=ggiaX7df1Dwxp9OSeib4Kv///w==kRfZ4qXiBtnyGub3tLGg99ynrcKy9d3hh3imyllomYGZ2D6cwY2HrIbo0ZfBl1x0AL5AgXh+5zBENAJn6OIU4ULKIqSj5WQ25/pJZIDRmFLPY6rdk80j2XJ5hqeZC6v0ZpqGylQc/KqWT1YrCk7wRqepcvEv+IeOHbRoAF+APJ+N4yI+8hVM5i2znBiZ2dqZU+dQqiKq7quEb2vxbodpUfH5MJg8U+QZiDfRxHATUoUyacwDid7KD1hMuCWI6V3Npwnu9aVKjF8pSko72FP0KwZThErdps54B1NIEzYJFdDEILsYAVr5hDdi7QffFlABx0kJfFEcHEFyFL1meMmmHyQJ9JYa8GHbFdGV27pk+JKJXJ8Edo3xzAKcgMr6VKoKAKPOlBLRdN/lU5UCQDgzvP1wue8N/uj0prUIYXStA3M0Fli0v2eYBaYAXWVV71figSiakFliA6PRPnaJ1LqgtLabtpKZLlnLZ+oXTkNyOhM2CuSMj86BwOs5f2/LK55/pKOaVzx3UxBLOxFy+++A47EnzKOUauAhdrXk+K4yMzy/3itIPs3o1giRK+EHhdGrnFSq6gBpHVTZJYCC0ojwdEWMuNZ54quPl77Lz4pJVgdI8ncKXU0jEeN7Sby4xIOxugz8Hk3WdxW2v/Gaz4I3MQCHuqURWrGcurl/qa68jzaeBDv/w3dHatDh1CYJKFGJGjsdsGLzix8AsGlV98GD/nt9AUuUdWyAi3Q7gQaSKVQ5s9XW3+qm2B2x1U1UBAMBh+7JOgoSlVSV/mSbwes2+sSQYChbk3zkfDJJtCPx7p+mYzrrnbEQsy8Kt11PEZ61n6wKzfsKkEMazi1i606KbybfxHs480D2wQZcyx+uO60R15NIkpBEb4QuTZRxlkx2P#channelProductNo=6736074611'

### 🔧 이미지 링크
- 이미지 저장 기능 추가

In [216]:
# 상품 썸네일 이미지 링크
order_number = 0
product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src']

'https://searchad-phinf.pstatic.net/MjAyMzA1MDhfMjkg/MDAxNjgzNTEwNjQxNjYy.xUa6gZAGNXkMa56XZuYD7X7BtP6lz6-tG_pUDxJ-fZUg.HvspzRw0CZOdBfQveIE7os8pLB0vRDnAs39NaNl0-84g.PNG/2030179-742e2a42-b6d0-4cd8-98c4-839d3a9e89f5.png?type=f300'

### 상품 정보 섹션

In [217]:
# 상품 정보 섹션
order_number = 0
product_info_section = product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')
product_info_section

<div class="basicProductCardInformation_basic_product_card_information__7v_uc basicProductCardInformation_view_type_grid2__mlh6E" id="basic_product_card_information_6736074611"><div class="basicProductCardInformation_mall_area__xcGLH"><div class="basicProductCardInformation_advertisement_area__HzaQ_"><span class="advertisementTag_advertisement_tag__yIvim"><a class="advertisementTag_link__NAi19 _nlog_click" data-shp-area="prod.adinfo" data-shp-area-id="adinfo" data-shp-area-type="action" data-shp-inventory="prod" data-shp-nsc="svc.shopping.plusall" data-shp-page-key="100407255" data-shp-sti="shopping" href="#" role="button" target="_blank"><svg class="advertisementTag_icon__DdazQ advertisementTag_information__jKJtn" fill="none" height="18" viewbox="0 0 40 18" width="40" xmlns="http://www.w3.org/2000/svg"><rect height="17" rx="8.5" stroke="#D9D9DC" width="39" x="0.5" y="0.5"></rect><path d="M13.096 11.988c0 1.133-1.265 1.826-3.333 1.826s-3.311-.693-3.311-1.826 1.254-1.837 3.311-1.837 3.3

#### 상품 제목

In [218]:
# 상품 제목
print(product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text)

메리쏘드 릴타투 브러쉬 펜라이너 붓펜 아이라인


#### 🔧 상품 가격
- 숫자만 추출

In [219]:
# 상품 가격

print(product_info_section.select_one('span.priceTag_original_price__jyZRY').text)
print(product_info_section.select_one('span.priceTag_discount__F_ZXz').text)
print(product_info_section.select_one('span.priceTag_inner_price__TctbK').text)

할인 전 판매가11,000원
5%할인
10,400원


#### 🔧 배송비
- 숫자만 추출

In [220]:
product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text

'배송비 3,000원'

#### 종합

In [221]:
order_number = 0

# 상세 페이지 링크
print(product_list[order_number].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href'])


# 이미지 링크
print(product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src'])

# 상품 정보 섹션
product_info_section = product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')

# 상품 제목
print(product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text)

# 원래 상품 가격
try:
    print(product_info_section.select_one('span.priceTag_original_price__jyZRY').text)
except:
    print('할인없음')
# 할인률
try:
    print(product_info_section.select_one('span.priceTag_discount__F_ZXz').text)
except:
    print('할인률없음')
# 현재 가격
print(product_info_section.select_one('span.priceTag_inner_price__TctbK').text)
# 배송비비
try:
    print(product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text)
except:
    print('배송비없음')

https://adcr.naver.com/adcr?x=ggiaX7df1Dwxp9OSeib4Kv///w==kRfZ4qXiBtnyGub3tLGg99ynrcKy9d3hh3imyllomYGZ2D6cwY2HrIbo0ZfBl1x0AL5AgXh+5zBENAJn6OIU4ULKIqSj5WQ25/pJZIDRmFLPY6rdk80j2XJ5hqeZC6v0ZpqGylQc/KqWT1YrCk7wRqepcvEv+IeOHbRoAF+APJ+N4yI+8hVM5i2znBiZ2dqZU+dQqiKq7quEb2vxbodpUfH5MJg8U+QZiDfRxHATUoUyacwDid7KD1hMuCWI6V3Npwnu9aVKjF8pSko72FP0KwZThErdps54B1NIEzYJFdDEILsYAVr5hDdi7QffFlABx0kJfFEcHEFyFL1meMmmHyQJ9JYa8GHbFdGV27pk+JKJXJ8Edo3xzAKcgMr6VKoKAKPOlBLRdN/lU5UCQDgzvP1wue8N/uj0prUIYXStA3M0Fli0v2eYBaYAXWVV71figSiakFliA6PRPnaJ1LqgtLabtpKZLlnLZ+oXTkNyOhM2CuSMj86BwOs5f2/LK55/pKOaVzx3UxBLOxFy+++A47EnzKOUauAhdrXk+K4yMzy/3itIPs3o1giRK+EHhdGrnFSq6gBpHVTZJYCC0ojwdEWMuNZ54quPl77Lz4pJVgdI8ncKXU0jEeN7Sby4xIOxugz8Hk3WdxW2v/Gaz4I3MQCHuqURWrGcurl/qa68jzaeBDv/w3dHatDh1CYJKFGJGjsdsGLzix8AsGlV98GD/nt9AUuUdWyAi3Q7gQaSKVQ5s9XW3+qm2B2x1U1UBAMBh+7JOgoSlVSV/mSbwes2+sSQYChbk3zkfDJJtCPx7p+mYzrrnbEQsy8Kt11PEZ61n6wKzfsKkEMazi1i606KbybfxHs480D2wQZcyx+uO60R15NIkpBEb4QuTZRxlkx2P#channelProductNo=6736074611
https://searchad

# 🔧 수집한 데이터 전처리
- 원래가격, 할인률, 현재가격 숫자만 표기하도록 정제
- 원래가격, 할인률 nan 값 처리된 부분 확인

In [222]:
num_list = []
detail_link_list = []
img_src_list = []
title_list = []
original_price_list = []
discount_rate_list = []
price_list = []
delivery_price_list = []

for i in range(len(product_list)):
    num = i + 1

    # 상세 페이지 링크
    try:
        detail_link = product_list[i].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href']
    except:
        print(i)
        # print('상세페이지링크없음')
    # 이미지 링크
    try:
        img_src = product_list[i].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src']
    except:
        print(i)
        # print('이미지링크없음')
    
    # 상품 정보 섹션
    product_info_section = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')
    
    ## 상품 제목
    try:
        title = product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text
    except:
        print(i)
        # print('상품제목없음')
    
    ## 원래가격
    try:
        original_price = product_info_section.select_one('span.priceTag_original_price__jyZRY').text
    except:
        original_price = np.nan
        # print('원래가격없음')
    ## 할인률
    try:
        discount_rate = product_info_section.select_one('span.priceTag_discount__F_ZXz').text
    except:
        discount_rate = np.nan
        # print('할인률없음')
    ## 현재가격
    try:
        price = product_info_section.select_one('span.priceTag_inner_price__TctbK').text
    except:
        print(i)
        # print('현재가격없음')
    ## 배송비
    try:
        delivery_price = product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text
    except:
        delivery_price = np.nan
        # print('배송비없음')

    # 정보 리스트에 담기
    num_list.append(num)
    detail_link_list.append(detail_link)
    img_src_list.append(img_src)
    title_list.append(title)
    original_price_list.append(original_price)
    discount_rate_list.append(discount_rate)
    delivery_price_list.append(delivery_price)
    price_list.append(price)


20
21
23
30
71
72
74
80
81
82
88
98


In [223]:
len(num_list), len(detail_link_list), len(img_src_list), len(title_list), len(original_price_list), len(discount_rate_list), len(price_list), len(delivery_price_list)

(99, 99, 99, 99, 99, 99, 99, 99)

In [224]:
data_dict = {
    '번호':num_list, 
    '상세페이지링크':detail_link_list,
    '이미지링크':img_src_list,
    '제품제목':title_list,
    '원래가격':original_price_list,
    '할인률':discount_rate_list,
    '현재가격':price_list,
    '배송비':delivery_price_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비
0,1,https://adcr.naver.com/adcr?x=ggiaX7df1Dwxp9OS...,https://searchad-phinf.pstatic.net/MjAyMzA1MDh...,메리쏘드 릴타투 브러쉬 펜라이너 붓펜 아이라인,"할인 전 판매가11,000원",5%할인,"10,400원","배송비 3,000원"
1,2,https://adcr.naver.com/adcr?x=jATOuDTTywenDV6m...,https://shopping-phinf.pstatic.net/main_825922...,[공식]메이블린 라인 타투 아이 라이너 크레용,"할인 전 판매가13,000원",25%할인,"9,750원",NaN
2,3,https://adcr.naver.com/adcr?x=tHd5bFMh+Lfh+Bbf...,https://shopping-phinf.pstatic.net/main_830583...,번지지않는 붓펜 아이라이너 점막 안번지는 저자극 그리기쉬운 아이라인 펜슬 붓 트임,"할인 전 판매가12,900원",23%할인,"9,900원","배송비 3,000원"
3,4,https://adcr.naver.com/adcr?x=Up/GXCxSPD4M4JlT...,https://shopping-phinf.pstatic.net/main_880392...,메이블린 뉴욕 라인 타투 크레용 0.4g 다크브라운 2개,"할인 전 판매가21,740원",10%할인,"19,560원","배송비 3,500원"
4,5,https://smartstore.naver.com/main/products/551...,https://shopping-phinf.pstatic.net/main_830583...,번지지않는 붓펜 아이라이너 점막 안번지는 저자극 그리기쉬운 아이라인 펜슬 붓 트임,"할인 전 판매가12,900원",23%할인,"9,900원","배송비 3,000원"


In [225]:
df.tail()

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비
94,95,https://smartstore.naver.com/main/products/111...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",선명하고 진한눈매 블랙아이라인 붓펜타입 아이라이너,"할인 전 판매가11,700원",10%할인,"10,530원","배송비 3,000원"
95,96,https://smartstore.naver.com/main/products/108...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",판다디자인몰 아이라인 가드 아이가드,"할인 전 판매가1,400원",20%할인,"1,120원","배송비 3,000원"
96,97,https://smartstore.naver.com/main/products/658...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",라이너브러시 아이라인그리기 실리콘 아이브러쉬,"할인 전 판매가24,910원",12%할인,"21,920원","배송비 3,000원"
97,98,https://smartstore.naver.com/main/products/109...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",메이크업 소품 아이라인 그리기 마스카라 가드 바르기 속눈썹 붙이기 도구,"할인 전 판매가2,290원",3%할인,"2,220원","배송비 3,000원"
98,99,https://smartstore.naver.com/main/products/100...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",점막 아이라이너 붓펜 워터프루프 아이라인 펜슬,NaN,NaN,"11,900원","배송비 3,000원"


In [226]:
df.loc[df['현재가격'].isnull()]

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비


In [227]:
df.loc[df['배송비'].isnull()]

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비
1,2,https://adcr.naver.com/adcr?x=jATOuDTTywenDV6m...,https://shopping-phinf.pstatic.net/main_825922...,[공식]메이블린 라인 타투 아이 라이너 크레용,"할인 전 판매가13,000원",25%할인,"9,750원",NaN
5,6,https://smartstore.naver.com/main/products/906...,https://shopping-phinf.pstatic.net/main_866077...,잘안지워지는 점막 지워지지않는 수영장 음영 얇은 아이라인 아이라이너 펜슬,"할인 전 판매가17,800원",11%할인,"15,800원",NaN
6,7,https://smartstore.naver.com/main/products/399...,https://shopping-phinf.pstatic.net/main_978286...,1+1 리르 룩스 젤 펜슬 아이라이너 아이라인 아이메이크업 0.5g,"할인 전 판매가20,000원",50%할인,"9,900원",NaN
12,13,https://smartstore.naver.com/main/products/467...,https://shopping-phinf.pstatic.net/main_822170...,아임유니 울트라 슬림 워터프루프 젤 아이라이너 펜슬 8컬러 안번지는 점막 아이라인 컬러,"할인 전 판매가11,000원",46%할인,"5,900원",NaN
14,15,https://smartstore.naver.com/main/products/504...,https://shopping-phinf.pstatic.net/main_825922...,메이블린 뉴욕 라인 타투 아이 라이너 크레용 0.4g,"할인 전 판매가13,000원",25%할인,"9,750원",NaN
23,24,https://smartstore.naver.com/main/products/562...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",블레스문 블랙홀 라스팅 워터프루프 아이라이너 (3Colors) 라인괴물 지속력 끝판...,NaN,NaN,"24,000원",NaN
34,35,https://smartstore.naver.com/main/products/502...,https://shopping-phinf.pstatic.net/main_825677...,1+1미크릿 비건 안번지는 미온수클렌징 아이라이너 0.6g(다크브라운/블랙),"할인 전 판매가24,000원",31%할인,"16,500원",NaN
39,40,https://smartstore.naver.com/main/products/733...,https://shopping-phinf.pstatic.net/main_848769...,헤라 이지 스타일링 아이라이너 0.5g 붓펜 아이라인 워터프루프,NaN,NaN,"21,780원",NaN
42,43,https://smartstore.naver.com/main/products/455...,https://shopping-phinf.pstatic.net/main_821019...,(3개)비러브 마이라인 펜 아이라이너 블랙,"할인 전 판매가11,400원",9%할인,"10,370원",NaN
57,58,https://smartstore.naver.com/main/products/905...,https://shopping-phinf.pstatic.net/main_866019...,점막 잘 안지워지는 지워지지않는 아이라인 브라운 컬러 얇은 무쌍 아이라이너,"할인 전 판매가29,800원",46%할인,"15,800원",NaN


In [228]:
sum(df['배송비'].isnull())

16

# 데이터 출력

In [229]:
current_date = time.strftime('%Y%m%d')
current_date

'20241206'

In [230]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241206_PM_080743'

In [231]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [233]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\20241206'

In [234]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [235]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_{search_keyword}_{current_datetime}.xlsx'), index=False)

# END